# Grain Data 

To study the *microstructure-properties* relationship for polycrystalline materials, collecting morphological and physical properties of the grains constituting a microstructure is an important task, that complements the study of microstructure images. This tutorial will review how to store, load and process grain data with *Pymicro*.

In the `Microstructure` class data model, the `GrainData` group is aimed at storing statistical data describing the sample grains, that are mostly stored within a structured array, the **GrainDataTable**. 

One of Pymicro's example datasets will be used to study this group. Let's start by opening it with the `Microstructure` class and display the content of this group:

In [ ]:
from pymicro import get_examples_data_dir # import file directory path
PYMICRO_EXAMPLES_DATA_DIR = get_examples_data_dir() # get the file directory path
import os 

# import Microstructure class
from pymicro.crystal.microstructure import Microstructure 
micro = Microstructure(filename=os.path.join(PYMICRO_EXAMPLES_DATA_DIR,'t5_dct_slice_data.h5'))

# display CellData group content
micro.print_node_info('GrainData')
micro.print_group_content('GrainData', short=True)

## The Grain Data Table

The `GrainDataTable` mentioned above is the only data item stored in the `GrainData` group in the standard data model of the `Microstructure` class. Let's take a look at its content:

In [ ]:
print(type(micro['GrainDataTable']))
print(micro['GrainDataTable'].dtype)
print(micro['GrainDataTable'])

<a id='l1'></a>
### Get values from the GrainDataTable

The `GrainDataTable` stores a structured table, that can be retrieved as a *Numpy* structured array. As shown below, its fields provide the following information on the sample grains:

* an identity number of the grain
* two columns describing the grain geometry: volume and center (position of center of mass in sample)
* the orientation of the grain provided as a **Rodrigues** vector
* the indices of the grain bounding box in the `CellData` image field arrays

To retrieve those values, you can use standard manipulation commands for `SampleData` structured tables and numpy arrays (see [dedicated tutorial](./Data_Items.ipynb)), or dedicated `Microstructure` class methods, such as `get_grain_centers`:

* `get_grain_centers`
* `get_grain_bounding_boxes`
* `get_grain_volumes`

In [ ]:
import numpy as np

# retrieve table as numpy structured array with SampleData dictionary like access
GrainDataTable = micro['GrainDataTable']

# get table columns from class methods and compare to arrays got with numpy array manipulation
grain_ids = micro.get_grain_ids()
print(f'grain ids equal ? {np.all(grain_ids == GrainDataTable["idnumber"])}')
grain_centers = micro.get_grain_centers()
print(f'grain centers equal ? {np.all(grain_centers == GrainDataTable["center"])}')
grain_volumes = micro.get_grain_volumes()
print(f'grain volumes equal ? {np.all(grain_volumes == GrainDataTable["volume"])}')
grain_bboxes = micro.get_grain_bounding_boxes()
print(f'grain bounding boxes equal ? {np.all(grain_bboxes == GrainDataTable["bounding_box"])}')
grain_rodrigues = micro.get_grain_rodrigues()
print(f'grain orientations equal ? {np.all(grain_rodrigues == GrainDataTable["orientation"])}')

Other methods to get specific grain data are also available in the class interface:

In [ ]:
centers = micro.get_grain_positions()
print(f'The position of the 10 first grain centers of mass are:\n {centers[:10]}\n')

volume_fractions = micro.get_grain_volume_fractions()
print(f'The 10 first grain volume fractions are:\n {volume_fractions[:10]}\n')

volume_fr = micro.get_grain_volume_fraction(18)
print(f'Volume fraction of grain 18 is {volume_fr*100:1.3f}%')

### The grains attribute

The `Microstructure.grains` attribute is an alias for the *GrainDataTable* data item. As such, it allows to manipulate and interact directly with the *GrainDataTable*. You can use it to access grain data just like you would manipulate a *Numpy* structured array: 

In [ ]:
print(micro.grains[0]['center'],'\n')
print(micro.grains[4:10]['orientation'])

Hence, getting an information on a grain whose id number is known, can be done with a call of the form `micro.grains["idnumber"]['"information"']`.

### Iterate Grains

The `GrainDataTable` can also be iterated. The iteration is done row by row of the structured array, and the iterator return these rows during a for loop:  

In [ ]:
# iterate through grains with ID number below 200 and print center of masss
for g in micro.grains:
    if g["idnumber"] > 100:
        break
    print(f'Grain {g["idnumber"]} center of mass is located at {g["center"]}')

These row object can be manipulated just like *Numpy* structured arrays. They have the same fields as the `GrainDataTable`.

## Grain Objects

*Pymicro* also has *Grain* objects that are specific containers equivalent to a row of the dataset *GrainDataTable*. You can get them with the following methods: 

In [ ]:
# get the grain object of a specific grain
grain = micro.get_grain(18)
print(f'Grain 18 grain object:\n {grain}')

**As you can see, a `Grain` object is essentially defined by its identity number and is crystal orientation**. In addition, it also stores the position of the grain (`center` attribute).

The `Grain` class provides methods to get physical or crystallographic information on the grain (volume, Schmid factor, Bragg condition, orientation...). For instance, to compute the Schmid factor of the grain for a given slip system, you can proceed as follows:

In [ ]:
# get the lattice of the sample phase
lattice = micro.get_phase().get_lattice()

# get Schmid factor of second grain in microstructure for first basal slip system
grain_id = GrainDataTable['idnumber'][1]
grain = micro.get_grain(grain_id)
Schmid = grain.schmid_factor(lattice.get_slip_systems('basal')[0]) 
print(f'Schmid factor of grain {grain.id} for first basal slip system is {Schmid:1.3f}')

# get Schmid factor of grain 18 for third prismatic slip system
grain = micro.get_grain(18)
Schmid = grain.schmid_factor(lattice.get_slip_systems('prism')[2]) 
print(f'Schmid factor of grain {grain.id} for first basal slip system is {Schmid:1.3f}')

You may also get a list of grain objects that includes all grains in the dataset, with the `get_all_grains` method:

In [ ]:
# get a list of all grain objects in the microstructure
grains_list = micro.get_all_grains()
print(f'First 2 grain objects of the microstructure:\n {grains_list[:2]}')

## Set GrainDataTable values

To conclude this tutorial, we will see how to add data into the `GrainDataTable` of a dataset. First, a new microstructure must be created to serve as an example:

In [ ]:
# create an empty Microstructure object
micro2 = Microstructure(filename='micro_test', autodelete=True, overwrite_hdf5=True)
# print content of grain data table
print(f'Current grain data table {micro2.grains}')

### Add grains to the Grain Data Table

New grains can be added to the `GrainDataTable`, it can be initialized from a list of grain orientations, with the `add_grains` method. To illustrate that, we will initialize the table of the new microstructure from the orientations of the example dataset:

In [ ]:
# get example dataset orientations
orientations = micro.grains[:]['orientation']

# count number of grains in the table
print(f'The microstructure has {micro2.get_number_of_grains()} grains')

# add new grains to new microstructure 
# orientations in GrainDataTable are stored with Rodrigues vector 
micro2.add_grains(orientation_list=orientations, orientation_type='rod')

# print content of grain data table
print(f'Current grain data table content (first 5 grains) \n {micro2.grains[:5]}')
print(f'Current grain ids (first 10 grains) \n {micro2.grains[:10]["idnumber"]}')

# count number of grains in the table
print(f'The microstructure has {micro2.get_number_of_grains()} grains')

As you can see 21 new grains have been created from the crystal orientation provided to the `add_grains` method. Their `idnumber` has been initialized from 0 to the n umber of added grains, and all other values (centers, volumes, bounding boxes) have been initialized to zero. If we repeat the operation, we will add again 21 grains to the microstructure, with the same orientations: 

In [ ]:
# count number of grains in the table
print(f'The microstructure has {micro2.get_number_of_grains()} grains')

# add new grains to new microstructure 
# orientations in GrainDataTable are stored with Rodrigues vector 
micro2.add_grains(orientation_list=orientations, orientation_type='rod')

# count number of grains in the table
print(f'The microstructure has now {micro2.get_number_of_grains()} grains')

### Set values from arrays

The methods to get values from the table, such as `get_grain_centers`, that have been shown [above](#l1) , have a counterpart to set these values, such as the `set_grain_centers`:

* `set_centers`
* `set_bounding_boxes`
* `set_volumes`

These methods allow to set all the values of a `GrainDataTable` column. Hence their input must have the appropriate shape:

In [ ]:
# set grain centers of new microstructure from centers of example dataset
centers = micro.get_grain_centers()

# new microstructure has new 42 grains 
# set new microstructure centers : 42 values must be passed 
micro2.set_centers(np.concatenate((centers, centers)))

# set grain centers of new microstructure from centers of example dataset
volumes = micro.get_grain_volumes()

# new microstructure has new 42 grains 
# set new microstructure centers : 42 values must be passed 
micro2.set_volumes(np.concatenate((volumes, volumes)))

# set grain centers of new microstructure from centers of example dataset
bbox = micro.get_grain_bounding_boxes()

# new microstructure has new 42 grains 
# set new microstructure centers : 42 values must be passed 
micro2.set_bounding_boxes(np.concatenate((bbox, bbox)))

In [ ]:
# print content of grain data table
print(f'Current grain data table \n {micro2.grains[:]}')

The `GrainDataTable` has been now completely filled, and can be used to compute statistics or for various grain related data processings. 

### Setting data for a specific grain

To change the data stored for a specific grain, you must iterate the table to find your grain object and set one of its values as if it was a *Numpy* structured array. Then you have to use the specific `update` method on the grain row to set the value in the dataset, as follows:

In [ ]:
# get old orientation value 
grain_orientation = micro2.GrainDataTable[15]['orientation']
print(f'The orientation of the grain is {micro2.GrainDataTable[15]["orientation"]}')

# iterate to find the grain and set its orientation to a random value 
for g in micro2.grains:
    if g['idnumber'] == 15:
        g['orientation'] = np.random.rand(3)
        g.update()
print(f'The new orientation of the grain is {micro2.GrainDataTable[15]["orientation"]}')

# Set back the original value of the orientation 
for g in micro2.grains:
    if g['idnumber'] == 15:
        g['orientation'] = grain_orientation
        g.update()

print(f'The orientation of the grain is back at {micro2.GrainDataTable[15]["orientation"]}')

In [ ]:
del micro2

### Setting grain data from the Grain Map

The `GrainDataTable` contains data describing the grains position and morphology. These values can be computed from the `grain map` (see [dedicated tutorial](./Cell_Data.ipynb)), that contains the information of the geometry of each grain. Specific methods of the `Microstucture` class allow to compute those values and automatically fill the `GrainDataTable` with them. They are:

* `recompute_grain_centers`: computes and fills the `center` column of the *GrainDataTable* from the grains geometry in grain map
* `recompute_grain_volumes`: computes and fills the `volume` column of the *GrainDataTable* from the grains geometry in grain map
* `recompute_grain_bounding_boxes`: computes and fills the `bounding_box` column of the *GrainDataTable* from the grains geometry in grain map

If you need to call them all, you can do it at once with the `build_grain_table_from_grain_map`, that will first synchronize the grain ids that are in the `grain map` and the `GrainDataTable`, and then call the 3 previous methods to fill the geometric grain data in the table. Note that the 

**Microstructure datasets have been designed to have the `GrainDataTable` and the `grain_map` synchronized. Try to keep consistent values in your datasets to use all Pymicro's functionalities.**  

This is an alternative way to initialize the `GrainDataTable` than the one shown above, that starts from grain orientations. This latter method is well suited for datasets created from imaging experiments.

Here is an example of out it can be done:

In [ ]:
# create an empty Microstructure object
micro2 = Microstructure(filename='micro_test', autodelete=True, overwrite_hdf5=True)
                        
# print content of grain data table
print(f'Current grain data table {micro2.grains} \n')

In [ ]:
# set the grain map from example dataset 
micro2.set_grain_map(micro.get_grain_map(), voxel_size=micro.get_attribute('spacing','CellData')[0])

# build the grain data table from the grain map
micro2.build_grain_table_from_grain_map()

# print new grain data table
print(micro2.GrainDataTable)

The table has been created with its `'center', 'bounding_box', 'volume'` columns that have been computed from the `grain_map`. The grain orientations have been initialized with a random value. To set user defined values, the `set_orientations` method can be used:

In [ ]:
# get orientations from example dataset
orientations = micro.get_grain_rodrigues()

# set orientations in new microstructure
micro2.set_orientations(orientations)

# print content of grain data table
print(micro2.GrainDataTable)

This concludes the tutorial on Pymicro's grain data !

In [ ]:
del micro2
del micro